In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.7]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [5]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.6203029755886177, 0.6059417882420914)

### Lasso Stumps

In [4]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_stumps").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_stumps").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [6]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.004, cols, 816)
single_stump_model['features']

['sex1',
 'age_at_current_charge18',
 'age_at_current_charge19',
 'age_at_current_charge20',
 'age_at_current_charge21',
 'age_at_current_charge27',
 'age_at_current_charge30',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_arrest7',
 'p_arrest8',
 'p_charges1',
 'p_charges5',
 'p_charges6',
 'p_violence1',
 'p_felony1',
 'p_misdemeanor1',
 'p_property1',
 'p_weapon1',
 'p_drug2',
 'p_drug3',
 'p_stealing1',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'five_year1',
 'current_violence1']

In [7]:
c_grid={'C': [0.001, 0.002, 0.003, 0.004]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.004}, {'C': 0.004}, {'C': 0.004}, {'C': 0.004}, {'C': 0.004}],
 0.6262599306658962,
 0.0004976388955342248)

### RiskSLIM

In [10]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

11

In [11]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [12]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'felony_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset = 100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
12/27/19 @ 11:05 PM | 163 rows in lookup table
12/27/19 @ 11:05 PM | ------------------------------------------------------------
12/27/19 @ 11:05 PM | runnning initialization procedure
12/27/19 @ 11:05 PM | ------------------------------------------------------------
12/27/19 @ 11:05 PM | CPA produced 2 cuts
12/27/19 @ 11:05 PM | running naive rounding on 5 solutions
12/27/19 @ 11:05 PM | best objective value: 0.6764
12/27/19 @ 11:05 PM | rounding produced 5 integer solutions
12/27/19 @ 11:05 PM | best objective value is 0.6931
12/27/19 @ 11:05 PM | running sequential rounding on 5 solutions
12/27/19 @ 11:05 PM | best objective value: 0.6764
12/27/19 @ 11:05 PM | sequential rounding produced 1 integer solutions
12/27/19 @ 11:05 PM | best objective value: 0.4596
12/27/19 @ 11:05 PM | polishing 6 solutions
12/27/19 @ 11:05 PM | best objective value: 0.4596
12/27/19 @ 11:05 PM | polishing produced 5 integer solutions
12/27/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:05 PM | adding 171 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3693.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:05 PM | adding 158 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3702.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:05 PM | adding 172 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3689.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:05 PM | adding 167 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3700.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:10 PM | adding 183 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3687.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:10 PM | adding 163 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3681.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:10 PM | adding 179 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3700.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:10 PM | adding 169 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3682.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/27/19 @ 11:11 PM | adding 178 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3687.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 26 columns, and 69 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [13]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.7258809370974342, 0.5967636862631808)

In [14]:
#### save results
summary_felony2_ky_inter_model = {"cart": cart_summary,
                                  "ebm": ebm_summary,
                                  "stumps": stumps_summary,
                                  "riskslim": riskslim_summary}

In [15]:
path = "./results/interpretable/two-year/"
results = [["Felony",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)